All paths as in properties.py but with changed UNIX style paths for the code to execute across platforms

In [1]:
import os 
V={}
WORK_DIR = '/media/hru/Data/_Active_Projects/PythonRDAHMM/test'
V['cron_path']=os.path.join(WORK_DIR, "RDAHMM","CRON_Download/")  
V['download_path']=os.path.join(WORK_DIR,"RDAHMM","Download/")  
V['script_path']=os.path.join(WORK_DIR,"PythonRDAHMM/") 
V['data_path']=os.path.join(WORK_DIR,"RDAHMM","Data/")
# temp_path is the temporary working directory for ingesting raw data
V['temp_path']=os.path.join(WORK_DIR,"RDAHMM","TEMP/")
V['model_path']=os.path.join(WORK_DIR,"RDAHMM","Model/")
V['eval_path']=os.path.join(WORK_DIR,"daily/single/")
V['train_epoch']="2013-12-31"
V['rdahmm_bin']=os.path.join(WORK_DIR,"RDAHMM", "rdahmm3","bin", "rdahmm")
V['rdahmm_model_parm']="-data <inputFile> -T <dataCount> -D <dimensionCount> -N 5 -output_type gauss -anneal -annealfactor 1.1 -betamin 0.1 -regularize -omega 0 0 1 1.0e-6 -ntries 10 -seed 1234"
V['rdahmm_eval_parm']="-data <proBaseName>.all.input -T <dataCount> -D <dimensionCount> -N 5 -output_type gauss -A <modelBaseName>.A -B <modelBaseName>.B -pi <modelBaseName>.pi -minvalfile <modelBaseName>.minval -maxvalfile <modelBaseName>.maxval -rangefile <modelBaseName>.range -eval"
V['dygraphsJs']=os.path.join(WORK_DIR,"PythonRDAHMM","dygraphsJsCreator.pearl")

def properties(key):
    return V[key]

# Generate the json file for Presentation layer
This is the existing code obtained from PythonRDAHMM/create_summary_jsons.py 

In [2]:
import os, sys, string, re, json
from datetime import date, datetime, timedelta, time

In [3]:
# command line input argument
# dataSet = "rawNeuTimeSeries.MEASURES_Combination"
dataSet = "UNR_SPLICE"

Note to set <b>endDate = '2016-10-19'</b> as the database was downloaded from gf9 that date

In [4]:
# Some useful global constants
today = datetime.today()
serverName = "gf9.ucs.indiana.edu"
updateTime = str(today.strftime("%Y-%m-%dT%H:%M:%S"))
beginDate = "1994-01-01"
# endDate = str(today.strftime("%Y-%m-%d"))
endDate = '2016-10-19'
centerLng = "-119.7713889"
centerLat = "36.7477778"
stateChangeNumTxtFile = "stateChangeNums.txt"
stateChangeNumJsInput = "stateChangeNums.txt.jsi"
allStationInputName = "all_stations.all.input"
filters = "Fill_Missing"

In [5]:
# Used to separate parts of the station name
SEPARATOR_CHARACTER="_"
NO_DATA_TIME_STAMP="22:22:22"
FINAL_PATH=properties('eval_path')

In [6]:
def setStationId(stationList, stationData):
    #Get the station name.
    stationName=stationList.split(SEPARATOR_CHARACTER)[2];

    stationData['id'] = stationName
    stationData['pro_dir'] = "daily_project_" + stationName + "_" + endDate
    stationData['AFile'] = "daily_project_" + stationName + ".A"
    stationData['BFile'] = "daily_project_" + stationName + ".B"
    stationData['InputFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input"
    stationData['RawInputFile'] = "daily_project_" + stationName + "_" + endDate + ".all.raw"
    stationData['SwfInputFile'] = "daily_project_" + stationName + "_" + endDate + ".plotswf.input"
    stationData['DygraphsInputFile'] = "daily_project_" + stationName + "_" + endDate + ".dygraphs.js"
    stationData['LFile'] = "daily_project_" + stationName + ".L"
    stationData['XPngFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input.X.png"
    stationData['YPngFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input.Y.png"
    stationData['ZPngFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input.Z.png"
    stationData['XTinyPngFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input.X_tiny.png"
    stationData['YTinyPngFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input.Y_tiny.png"
    stationData['ZTinyPngFile'] = "daily_project_" + stationName + "_" + endDate + ".all.input.Z_tiny.png"
    stationData['PiFile'] = "daily_project_" + stationName + ".pi"
    stationData['QFile'] = "daily_project_" + stationName + "_" + endDate + ".all.Q"
    stationData['MaxValFile'] = "daily_project_" + stationName + ".maxval"
    stationData['MinValFile'] = "daily_project_" + stationName + ".minval"
    stationData['RangeFile'] = "daily_project_" + stationName + ".range"
    stationData['ModelFiles'] = "daily_project_" + stationName + ".zip"
    stationData['RefFile'] = "daily_project_" + stationName + ".input.ref"
    return

In [7]:
def setStationStartDate(stationDir, stationData):
    startFileName = stationDir + "daily_project_" + stationData['id'] + ".input.starttime"
    if (os.path.isfile(startFileName)):
        with open(startFileName,"r") as startFile:
            startDate = startFile.readline().rstrip()
        startFile.close()
    else:
		startDate = "1994-01-01"
    stationData['start_date'] = startDate
    return


In [8]:
def setStationRefLatLonHgt(stationDir, stationData):
    refFileName = stationDir + stationData['RefFile']
    refLat=""
    refLon=""
    refHgt=""
    if (os.path.isfile(refFileName)):
        with open(refFileName,"r") as refFile:
            refParts=refFile.readline().split(" ")
            refLat=refParts[0]
            refLon=refParts[1]
            refHgt=refParts[2].rstrip() # Have to chomp off the final \n
        refFile.close()
    else:
        refLat="1.0"
        refLon="2.0"
        refHgt="-1.0"

    stationData['lat'] = refLat      
    stationData['long'] = refLon      
    stationData['height'] = refHgt
    return

In [9]:
def setStatusChanges(stationDir, stationData):
    # Open the .all.Q and the .all.raw files.  We get the state from the first and
    # the data from the second. 
    # TODO: for now, we assume these files always exist
    qFileName = stationDir + stationData['QFile']
    rawFileName = stationDir + stationData['RawInputFile']
   



    # Bail out if the required files don't exist   
    if((not os.path.isfile(qFileName)) or (not os.path.isfile(rawFileName))): 
        return 
    qFile = open(qFileName,"r")
    rawFile = open(rawFileName,"r")

    stateChanges = []
    changeCount = 0
    # Now step through the Q file looking for state changes
    # If we find a state change, get the date from the raw file
    # We will save these to the string stateChangeArray since we
    # need to record in latest-first order
    qline1 = qFile.readline()
    rline1 = rawFile.readline()        
    while True:
        eventData = {}
        qline2 = qFile.readline()
        rline2 = rawFile.readline()
        if not qline2: break
        
        # See if qline1 and qline2 are the same.  If so, extract the dates from rline1 and rline2
        # The line splits below are specific to the raw file line format.
        if (qline1.rstrip() != qline2.rstrip()):
            eventdate = rline2.split(" ")[1] 
            eventdate = eventdate.split("T")[0]
            oldstate = qline1.rstrip()
            newstate = qline2.rstrip()
            eventData['date'] = eventdate
            eventData['from'] = oldstate
            eventData['to'] = newstate 
            stateChanges.append(eventData)
            changeCount += 1

        # Make the previous "next" lines the "first" lines for the next comparison
        qline1=qline2
        rline1=rline2

    stationData['status_changes'] = stateChanges
    stationData['change_count'] = changeCount

    # Clean up
    qFile.close
    rawFile.close
    return

In [10]:
def setTimesNoData(stationDir, stationData):
    rawFileName = stationDir + stationData['RawInputFile']

    # Required file doesn't exist so bail out
    if(not os.path.isfile(rawFileName)): return
    rawFile = open(rawFileName, "r")
    
    noDataRanges = []
    noDataCount = 0
    noDataEvent = {}

    # We need to set a no-data range from beginDate (for the epoch, 1994-01-01) to the day before
    # our first data point for this station.  If the station has data before 1994-01-01, then 
    # ignore.
    firstDataDateParts=rawFile.readline().split(" ")[1].split("T")[0].split("-");

    beginEpoch=date(1994,1,1)

    #Convert this into a data object
    dayMinusOne=date(int(firstDataDateParts[0]),int(firstDataDateParts[1]),int(firstDataDateParts[2]))
    dayMinusOne-=timedelta(days=1)
    if(dayMinusOne > beginEpoch): 
        dayMinusOneString=dayMinusOne.isoformat()
        noDataEvent['to'] = dayMinusOneString
        noDataEvent['from'] = beginDate
        noDataCount += 1

    #Reset the "raw" file to the beginning
    rawFile.seek(0)

    # Step through the file to find the starting and ending dates with no data.
    # By convention, this occurs when the line has a timestamp T22:22:22.  Also, by
    # convention, we will record the latest to earliest dates with no data.

    while True:
        noDataEvent = {}
        nodata=False
        rline1=rawFile.readline()
        if not rline1: break

        # Get the date and timestamp, following format conventions
        fulleventdate1=rline1.split(" ")[1]
        eventdate1=fulleventdate1.split("T")[0]
        timestamp1=fulleventdate1.split("T")[1]

        # See if we have detected a no-data line
        if(timestamp1==NO_DATA_TIME_STAMP):
            nodata=True
            #Keep eventdate1 in case this is an isolated no-data line.
            eventdate_keep=eventdate1

            # We have a no-data line, so step ahead until the 
            # no-data line ends.
            while(nodata):
                rline2=rawFile.readline()
                if not rline2: break
                fulleventdate2=rline2.split(" ")[1]
                eventdate2=fulleventdate2.split("T")[0]
                timestamp2=fulleventdate2.split("T")[1]
                if(timestamp2!=NO_DATA_TIME_STAMP):
                    # Data exists for the second time stamp, so break out
                    # The last no-data line was the previous line
                    nodata=False
                    break
                else:
                    # No data for this line either, so keep this timestamp
                    # and start the while(nodata) loop again
                    eventdate_keep=eventdate2

            # We now know the range of no-data values, so insert this range, latest first
	    noDataEvent['to'] = eventdate_keep
	    noDataEvent['from'] = eventdate1
	    noDataRanges.append(noDataEvent)
	    noDataCount += 1
            
    # Finally, prepend the data-not-yet-available date range, from the last day of data
    # until today's date.
    today=date.today()
    formattedToday=today.isoformat() 
    
    #Reread the last event
    rawFile.seek(0)
    lastRawLine=rawFile.readlines()[-1]
    lastRawDate=lastRawLine.split(" ")[1].split("T")[0]
    lastDataDateParts=lastRawDate.split("-")  # This is the last date
    #Create a new date object out of the string we get from the file.
    lastDataDatePlus1=date(int(lastDataDateParts[0]),int(lastDataDateParts[1]),int(lastDataDateParts[2]))
    #Now increment this date one day.
    lastDataDatePlus1+=timedelta(days=1)    
    #Now convert to a string
    lastDataDataP1String=lastDataDatePlus1.isoformat()

    noDataEvent = {}
    noDataEvent['to'] = formattedToday
    noDataEvent['from'] = lastDataDataP1String
    noDataRanges.append(noDataEvent)
    noDataCount += 1
    
    stationData['time_nodata'] = noDataRanges
    stationData['nodata_count'] = noDataCount
    rawFile.close
    return

In [11]:
projectDir = FINAL_PATH +dataSet
if(os.path.isdir(projectDir)):
# Open the JSON file that will contain the results
    outputPath = FINAL_PATH+dataSet + "_FILL.json"
    summaryData = {}
        
    summaryData['update_time'] = updateTime
    summaryData['data_source'] = dataSet
    summaryData['begin_date'] = beginDate
    summaryData['end_date'] = endDate
    summaryData['center_longitude'] = centerLng
    summaryData['center_latitude'] = centerLat
    summaryData['server_url'] = "http://" + serverName + "/daily_rdahmmexec/daily/" + dataSet
    summaryData['stateChangeNumTxtFile'] = stateChangeNumTxtFile
    summaryData['stateChangeNumJsInput'] = stateChangeNumJsInput
    summaryData['allStationInputName'] = allStationInputName
    summaryData['Filters'] = filters
    summaryData['video_url'] = ""

    stations = []
    stationCount = 0

In [12]:
import os, errno

def silentremove(filename):
    try:
        os.remove(filename)
    except OSError as e: # this would be "except OSError, e:" before Python 2.6
        if e.errno != errno.ENOENT: # errno.ENOENT = no such file or directory
            raise # re-raise exception if a different error occured
            
silentremove(outputPath)

In [13]:
summaryData

{'Filters': 'Fill_Missing',
 'allStationInputName': 'all_stations.all.input',
 'begin_date': '1994-01-01',
 'center_latitude': '36.7477778',
 'center_longitude': '-119.7713889',
 'data_source': 'UNR_SPLICE',
 'end_date': '2016-10-19',
 'server_url': 'http://gf9.ucs.indiana.edu/daily_rdahmmexec/daily/UNR_SPLICE',
 'stateChangeNumJsInput': 'stateChangeNums.txt.jsi',
 'stateChangeNumTxtFile': 'stateChangeNums.txt',
 'update_time': '2016-12-27T19:12:50',
 'video_url': ''}

In [14]:
for stationList in os.listdir(projectDir):
    stationPath = projectDir + "/" + stationList + "/"
    if (os.path.isdir(stationPath)):
        stationData = {}

        setStationId(stationList, stationData)
        setStationStartDate(stationPath, stationData)
        setStationRefLatLonHgt(stationPath, stationData)
        setStatusChanges(stationPath, stationData)
        setTimesNoData(stationPath, stationData)

        stations.append(stationData)
        stationCount += 1 

summaryData['stations'] = stations
summaryData['station_count'] = stationCount

# with open(outputPath, 'w') as jsonfile:
#     jsonfile.write(json.dumps(summaryData, sort_keys=True, indent=2))
# jsonfile.close

# Integrate client side processing to store state changes in last days
This is python code translated from JavaScript code from https://github.com/GeoGateway/geogateway-portal/blob/master/html/js/gps-tools.js

In [15]:
import time

In [16]:
# global params
gpsStationState=["green","red","yellow","lightblue","blue"]

In [17]:
def convert_string_to_date(string):
#     return datetime.strptime(string, '%Y-%m-%d').now().date()
    return datetime.strptime(string, '%Y-%m-%d')

In [18]:
def checkDateForData(selectedDate,noDataDates):
    dataOnDate=True;
    #   Selected date is after the last no-data date.
    if selectedDate > convert_string_to_date(noDataDates[len(noDataDates) -1]['to']):
        dataOnDate=True;

    # Otherwise, check each no-data interval to see if the date falls within.
    else:
        for elem in noDataDates:
            startDate = convert_string_to_date(elem['from'])
            endDate = convert_string_to_date(elem['to'])

            if (startDate <= selectedDate) & (endDate >= selectedDate):
                dataOnDate=False
                break

    return dataOnDate

In [19]:
# Would be nice to throw an exception here 
def getPrecedingStateChange(selectedDate,statusChanges):
    
    stateLastDate = selectedDate
    latestPossibleDate=convert_string_to_date(statusChanges[len(statusChanges)-1]['date'])
    earliestPossibleDate=convert_string_to_date(statusChanges[0]['date'])

#     This should actually throw an erorr since there is no earlier state change.
    if(selectedDate <= earliestPossibleDate):
#         stateLastDate=earliestPossibleDate;
        stateLastDate=selectedDate
    
    elif (selectedDate >= latestPossibleDate):
        stateLastDate=latestPossibleDate
        
    else:
        for i, e in reversed(list(enumerate(statusChanges))):
            stateChangeDate1 = convert_string_to_date(statusChanges[i-1]['date'])
            stateChangeDate2 = convert_string_to_date(statusChanges[i]['date'])    
#             The last state change date to find is the one
#             on or before the curren date.
            if (selectedDate >= stateChangeDate1) & (selectedDate < stateChangeDate2):
                stateLastDate=stateChangeDate1
#                 Dates are in order, so we can stop
                break;
        
#         for(var i=statusChanges.length-1; i>0; i--) {
#             var stateChangeDate1=new Date(statusChanges[i-1].date);
#             var stateChangeDate2=new Date(statusChanges[i].date);
#             //The last state change date to find is the one
#             //on or before the curren date.
#             if(selectedDate >= stateChangeDate1 
#                && selectedDate < stateChangeDate2) {
#                 stateLastDate=stateChangeDate1;
#                 //Dates are in order, so we can stop
#                 break;
#             }
#         }
    return stateLastDate

In [20]:
def getStationState(date,gpsStation):
    #     theState=gpsStationState[0]  #This is the default.
    theState=0
    today=date
    lastMonth=today - timedelta(30)
    dayBefore=today - timedelta(1)

    statusChanges=gpsStation['status_changes']
    noDataDates=gpsStation['time_nodata']

    earliestDataDate = convert_string_to_date(gpsStation['start_date'])
    dataOnDate = checkDateForData(date,noDataDates)

    #     Hopefully this big if-else construction correctly captures all the case.
    #     It can be simplified later.

    #     Provided date is before any data available for that station, so state is light blue
    if (today < earliestDataDate):
    #         theState=gpsStationState[3]  #light blue
        theState=3
    #     print ("c1")


    #     Date falls within data range, there are no status changes, and data is available on date
    elif (today >= earliestDataDate) & (len(statusChanges)==0) & (dataOnDate==True):
    #         theState=gpsStationState[0]  # green
        theState=0
    #     print ("c2")

    #     Date falls within data range, there are no state changes, and no data on selected date.
    elif (today > earliestDataDate) & (len(statusChanges)==0) &(dataOnDate==False):
    #         theState=gpsStationState[3]  # light blue
        theState=3
    #     print ("c3")


    #     We have data on the date, but it proceeds the earliest state change
    elif (today < convert_string_to_date(statusChanges[0]['date'])):
    #         theState=gpsStationState[0]  # green
        theState=0
    #     print ("c4")

    #     See if the date falls within 1 day or 1 month of a state change.
    elif (len(statusChanges) > 0):
        #     print ("c5")
        #    Get nearest preceding state change date
        stateLastDate=getPrecedingStateChange(date,statusChanges)

    #         See if state change was yesterday.
        if(stateLastDate > dayBefore):
            #             theState=gpsStationState[1]  # red
            theState=1

    #         See if the station has changed state in between the
    #         selected date and 30 days prior to the selected date.
        elif (stateLastDate >= lastMonth) & (stateLastDate <= today):
            #       See if we have no data within 24 hours of the selected date.            
            if (dataOnDate == False):
                #       Data is missing within last 24 hours and state has changed within last 30 days.
                #                 theState=gpsStationState[4]  # blue
                theState=4
            else:
                #       We have data on the date and state has changed within a 30 day window.
                #                 theState=gpsStationState[2] # yellow
                theState=2
        #         No data is available on selected date for this station, and station is
        #         not within either the 1 day or 30 day window.
        elif(dataOnDate==False):
            #             theState=gpsStationState[3]  # light blue
            theState=3

    return theState

## SAVE TO MONGODB

In [21]:
import pymongo
from pymongo import MongoClient

In [47]:
client = MongoClient('localhost', 27017)

In [53]:
client.database_names()

[u'GPS_mXn_UNR_SPLICE', u'local', u'poetry_london']

In [49]:
dataSet = 'UNR_SPLICE'
database_name='GPS_'+dataSet
database_name

'GPS_UNR_SPLICE'

In [25]:
# def daterange(start_date, end_date):
# #     http://stackoverflow.com/questions/1060279/iterating-through-a-range-of-dates-in-python?answertab=votes#tab-top
#     for n in range(int ((end_date - start_date).days)):
#         yield start_date + timedelta(n)

# start_date = datetime.strptime(beginDate, '%Y-%m-%d')
# end_date = datetime.strptime(endDate, '%Y-%m-%d')

# for date in daterange(start_date, end_date):
#     for station in stations:
#         break
#         #print date.strftime('%Y-%m-%d') + " : " + str(getStationState(date, station))
# #     print type(date)

## Drop existing database and create new database

In [52]:
client.drop_database(database_name)
db =client[database_name]

In [27]:
# Create 3 collections
# for network meta data
collections_meta_network = db.collections_meta_network
# for station meta data
collections_meta_stations = db.collections_meta_stations
# for stations
collections_time_series_stations = db.collections_time_series_stations 

### Create network-meta

In [28]:
meta_network= {}
meta_network['update_time'] = summaryData['update_time']
meta_network['data_source'] = summaryData['data_source']
meta_network['begin_date'] = summaryData['begin_date']
meta_network['end_date'] = summaryData['end_date']
meta_network['center_longitude'] = summaryData['center_longitude']
meta_network['center_latitude'] = summaryData['center_latitude']
meta_network['server_url'] = summaryData['server_url']
meta_network['stateChangeNumTxtFile'] = summaryData['stateChangeNumTxtFile']
meta_network['stateChangeNumJsInput'] = summaryData['stateChangeNumJsInput']
meta_network['allStationInputName'] = summaryData['allStationInputName']
meta_network['Filters'] = summaryData['Filters']
meta_network['video_url'] = summaryData['video_url']
meta_network['station_count'] = summaryData['station_count']

collections_meta_network.insert_one(meta_network)

### Create stations and stations-meta 

In [29]:
def get_legacy_data(station):
    
    document= {}
    document['_id'] = station['id']
    document['pro_dir'] = station['pro_dir']
    document['AFile'] = station['AFile']
    document['BFile'] = station['BFile']
    document['InputFile'] = station['InputFile']
    document['RawInputFile'] = station['RawInputFile']
    document['SwfInputFile'] = station['SwfInputFile']
    document['DygraphsInputFile'] = station['DygraphsInputFile']
    document['LFile'] = station['LFile']
    document['XPngFile'] = station['XPngFile']
    document['YPngFile'] = station['YPngFile']
    document['ZPngFile'] = station['ZPngFile']
    document['XTinyPngFile'] = station['XTinyPngFile']
    document['YTinyPngFile'] = station['YTinyPngFile']
    document['ZTinyPngFile'] = station['ZTinyPngFile']
    document['PiFile'] = station['PiFile']
    document['QFile'] = station['QFile']
    document['MaxValFile'] = station['MaxValFile']
    document['MinValFile'] = station['MinValFile']
    document['RangeFile'] = station['RangeFile']
    document['ModelFiles'] = station['ModelFiles']
    document['RefFile'] = station['RefFile']
    document['start_date'] = station['start_date']
    document['lat'] = station['lat']
    document['long'] = station['long']
    document['height'] = station['height']
    
    return document

In [30]:
start_date=datetime.strptime(beginDate, '%Y-%m-%d')
end_date=datetime.strptime(endDate, '%Y-%m-%d')

In [32]:
# http://stackoverflow.com/questions/21231789/how-to-get-all-days-in-current-month
from calendar import monthrange

# temp_station_list= []
for station in stations:
#     GeoJson format: maybe considered in future
#     loc = {'type' : "Point", 
#            'coordinates' : [float(station['long']), float(station['lat'])]
#           }
    loc = [float(station['long']), float(station['lat'])]   
    document={'station_id' : station['id'], 'loc' : loc }
    
    data_for_all_years = {}
    
    for year in range(start_date.year,end_date.year+1):
        data_for_a_year = {}
        
        for month in range(1,13):
            no_of_days_in_month = monthrange(year, month)[1]+1
            days=range(1, no_of_days_in_month)
#             state=[None]*(no_of_days_in_month-1)          
            data_for_a_month={}
            
            for day in days:
                date_in_time_series=datetime(year, month, day)
                
                if (date_in_time_series >= end_date):
                    data_for_a_year[str(month)] = data_for_a_month
                    data_for_all_years[str(year)] = data_for_a_year
                    break                
#                 state[str(day-1)]=str(getStationState(date_in_time_series, station))
                data_for_a_month[str(day)] = str(getStationState(date_in_time_series, station))
#             data_for_a_year[str(month)] = dict(zip(days, state))

            if (date_in_time_series >= end_date):
                break
            data_for_a_year[str(month)] = data_for_a_month
            
        if (date_in_time_series >= end_date):
            break            
        data_for_all_years[str(year)] = data_for_a_year
        
    document['status'] = data_for_all_years
#     set_legacy_data(document, station)
    
    # Add station time series
    collections_time_series_stations.insert_one(document)
    
    # Add station metadata
    collections_meta_stations.insert_one(get_legacy_data(station))
#     temp_station_list.append(document)

In [33]:
get_legacy_data(station)
# document

{'AFile': 'daily_project_ESM1.A',
 'BFile': 'daily_project_ESM1.B',
 'DygraphsInputFile': 'daily_project_ESM1_2016-10-19.dygraphs.js',
 'InputFile': 'daily_project_ESM1_2016-10-19.all.input',
 'LFile': 'daily_project_ESM1.L',
 'MaxValFile': 'daily_project_ESM1.maxval',
 'MinValFile': 'daily_project_ESM1.minval',
 'ModelFiles': 'daily_project_ESM1.zip',
 'PiFile': 'daily_project_ESM1.pi',
 'QFile': 'daily_project_ESM1_2016-10-19.all.Q',
 'RangeFile': 'daily_project_ESM1.range',
 'RawInputFile': 'daily_project_ESM1_2016-10-19.all.raw',
 'RefFile': 'daily_project_ESM1.input.ref',
 'SwfInputFile': 'daily_project_ESM1_2016-10-19.plotswf.input',
 'XPngFile': 'daily_project_ESM1_2016-10-19.all.input.X.png',
 'XTinyPngFile': 'daily_project_ESM1_2016-10-19.all.input.X_tiny.png',
 'YPngFile': 'daily_project_ESM1_2016-10-19.all.input.Y.png',
 'YTinyPngFile': 'daily_project_ESM1_2016-10-19.all.input.Y_tiny.png',
 'ZPngFile': 'daily_project_ESM1_2016-10-19.all.input.Z.png',
 'ZTinyPngFile': 'daily_

In [34]:
date_in_time_series

datetime.datetime(2016, 10, 20, 0, 0)

In [35]:
# temp_mySummary['stations'] = temp_station_list

In [36]:
# outputPath = properties('eval_path') + 'my_UNR_SPLICE_FILL.json'
# with open(outputPath, 'w') as jsonfile:
#     jsonfile.write(json.dumps(temp_mySummary, sort_keys=True, indent=2))

In [37]:
# Add station data
# collections_stations.insert_many(temp_station_list)

In [38]:
# Create index based on latitude, longitude
# https://docs.mongodb.com/v3.2/tutorial/build-a-2d-index/
# db.<collection>.createIndex( {<location field> : "<index type>"} ,
#                              { bits : <bit precision> } )

db.collections_time_series_stations.create_index( [("lon", pymongo.GEO2D)] )

u'lon_2d'

## Query DB

### Network data

In [39]:
output = {}
cursor_meta_network = db.collections_meta_network.find()

for i in cursor_meta_network:
    i.pop('_id', None)
    i['network_station_count'] = i['station_count']
    i.pop('station_count', None)
    output = i
    
    
print output

{u'update_time': u'2016-12-27T19:12:50', u'data_source': u'UNR_SPLICE', u'begin_date': u'1994-01-01', u'video_url': u'', u'end_date': u'2016-10-19', u'allStationInputName': u'all_stations.all.input', u'stateChangeNumTxtFile': u'stateChangeNums.txt', u'center_latitude': u'36.7477778', u'center_longitude': u'-119.7713889', 'network_station_count': 5, u'server_url': u'http://gf9.ucs.indiana.edu/daily_rdahmmexec/daily/UNR_SPLICE', u'Filters': u'Fill_Missing', u'stateChangeNumJsInput': u'stateChangeNums.txt.jsi'}


In [40]:
try:
	for i in db.collections_meta_network.find():
	    end_date = i['end_date']
except:
	end_date = datetime.today().strftime("%Y-%m-%d")
    
end_date

u'2016-10-19'

In [41]:
for i in db.collections_meta_network.find():
    print i

{u'update_time': u'2016-12-27T19:12:50', u'data_source': u'UNR_SPLICE', u'begin_date': u'1994-01-01', u'video_url': u'', u'end_date': u'2016-10-19', u'allStationInputName': u'all_stations.all.input', u'stateChangeNumTxtFile': u'stateChangeNums.txt', u'center_latitude': u'36.7477778', u'center_longitude': u'-119.7713889', u'server_url': u'http://gf9.ucs.indiana.edu/daily_rdahmmexec/daily/UNR_SPLICE', u'Filters': u'Fill_Missing', u'stateChangeNumJsInput': u'stateChangeNums.txt.jsi', u'_id': ObjectId('586303848bf0362de64900fe'), u'station_count': 5}


### Data for 1 station

In [42]:
station_id_to_find = 'ESM1'

output = {}
cursor_meta_stations = db.collections_meta_stations.find( \
                                        { '_id': station_id_to_find })

print cursor_meta_stations.count()

for i in cursor_meta_stations:
    output=i 
    
dygraph_file_path = os.path.join(properties('eval_path'),dataSet, \
                    'daily_project_'+ station_id_to_find+'_'+ end_date)

for file in os.listdir(dygraph_file_path):
    if file.endswith(".js"):
        with open(os.path.join(dygraph_file_path, file), 'r') as js_file:
            dygraphs=js_file.readlines()
#         with open("Output.js", "w") as text_file:
#             text_file.write("{}".format(data))

1


In [43]:
cursor_meta_stations

 #### Hack to get around frontend bug 
 JavaScript at fronend is unable to process the long string for dygraph. It somehow drops some of the text. So dividing the text into parts and sending them to frontend, then joining these strings to form a single string 

In [44]:
dy_list = dygraphs.split('}')
output['d_0'] = dy_list[0]
output['d_1'] = dy_list[1]
output['d_2'] = dy_list[2]
output['d_3'] = dy_list[3]
output['d_4'] = dy_list[4]
output['d_5'] = dy_list[5]

dy_list

AttributeError: 'list' object has no attribute 'split'

### Get status of stations within bounding box for particular date 

In [50]:
lat_min = -89.998914447
lat_max = 83.64323665
lon_min = -359.998835383
lon_max = -0.037155605

year = '2016'
month = '10'
day = '2'

# cursor_stations = collections_stations.find( {"lat" : {"$gte" : lat_min, "$lte" : lat_max}, \
#                             "long" : {"$gte" : long_min, "$lte" : long_max}})

# db.<collection>.find( { loc: [ <x> , <y> ] } )

# cursor_stations = collections_stations.find({"loc" : [long_min, lat_min]})

# db.places.find( { loc :
#                   { $geoWithin :
#                      { $box : [ [ 0 , 0 ] ,
#                                 [ 100 , 100 ] ]
#                  } } } )

status_to_find = 'status.' + year + '.' + month + '.' + day
cursor_stations = db.collections_time_series_stations.find({ "loc" : \
                                                  { "$geoWithin" : 
                                                    { "$box" : [ [lon_min, lat_min],\
                                                               [lon_max, lat_max]] \
                                                    } }, \
                                                 status_to_find : {'$exists': 1} \
                                                }, {'station_id': 1, 'loc': 1, status_to_find:1})
# http://stackoverflow.com/questions/20662691/how-to-search-through-a-mongodb-collection-for-dictionary-keys-nested-in-array
# http://stackoverflow.com/questions/5301795/mongodb-get-specific-part-of-document

list_station = []
if cursor_stations:
    for station in cursor_stations:
#         station.pop('_id', None)
#         list_station.append(station)
        try:
            res = {'station_id' : station['station_id'],
                   'status' : station['status'][year][month][day],
                   'lat' : station['loc'][1],
                   'lon' : station['loc'][0]
                  }
            list_station.append(res)
        except:
            continue

In [51]:
list_station

[{'lat': -12.466640097,
  'lon': -229.156012745,
  'station_id': u'00NA',
  'status': u'3'},
 {'lat': -12.478223997,
  'lon': -229.017953041,
  'station_id': u'01NA',
  'status': u'3'},
 {'lat': 30.40742467,
  'lon': -91.180261768,
  'station_id': u'1LSU',
  'status': u'3'},
 {'lat': 31.75080049,
  'lon': -93.097603682,
  'station_id': u'1NSU',
  'status': u'0'},
 {'lat': 47.803480434,
  'lon': -122.569173848,
  'station_id': u'ESM1',
  'status': u'3'}]

# MONGODB mXn

## Add data to database

In [ ]:
dataSet = 'UNR_SPLICE'
database_name='GPS_mXn_'+dataSet
database_name

In [ ]:
# Create database
client.drop_database(database_name)
db =client[database_name]

In [ ]:
# Create 2 collections
# for stations
collections_stations = db.collections_stations

In [ ]:
start_date=datetime.strptime(beginDate, '%Y-%m-%d')
end_date=datetime.strptime(endDate, '%Y-%m-%d')

In [ ]:
data = []
today = datetime.today()
for station in stations:
#     GeoJson format: maybe considered in future
#     loc = {'type' : "Point", 
#            'coordinates' : [float(station['long']), float(station['lat'])]
#           }
    loc = [float(station['long']), float(station['lat'])]   
    document={'station_id' : station['id'], 'loc' : loc }
        
    for year in range(start_date.year,end_date.year+1):
        
        for month in range(1,13):
            no_of_days_in_month = monthrange(year, month)[1]+1
            days=range(1, no_of_days_in_month)

            for day in days:
                date_in_time_series=datetime(year, month, day)
                if (date_in_time_series > today):
                    break
                
                doc = document.copy()
                doc['status'] = str(getStationState(date_in_time_series, station))             
                doc['date'] = date_in_time_series
                
                # Add station data
                data.append(doc)
                collections_stations.insert_one(doc)
            
            if (date_in_time_series > today):
                break
                
        if (date_in_time_series > today):
            break            

In [ ]:
date_in_time_series

In [ ]:
db.collections_stations.create_index([("loc", pymongo.GEO2D), \
                                        ("date", pymongo.ASCENDING)])

## Retrive data from mXn data

### Get status of stations within bounding box for particular date 

In [ ]:
lat_min = -89.998914447
lat_max = 83.64323665
lon_min = -359.998835383
lon_max = -0.037155605

year = '2013'
month = '12'
day = '12'

date_to_find_in_str = year + '-' + month + '-' + day
date_to_find = datetime.strptime(date_to_find_in_str,'%Y-%m-%d')

cursor_stations = db.collections_stations.find({ "loc" : \
                                                  { "$geoWithin" : 
                                                    { "$box" : [ [lon_min, lat_min],\
                                                               [lon_max, lat_max]] \
                                                    } }, \
                                                 "date" : {'$eq': date_to_find} \
                                                }, {'station_id': 1, 'status':1})

list_station = []
if cursor_stations:
    for record in cursor_stations:
#         print record
        record.pop('_id', None)
        list_station.append(record)
#         try:
#             res = {'id' : station['id'],
#                    'status' : station['status'][year][month][day]
#                   }
#             list_station.append(record)
#         except:
#             continue

In [ ]:
list_station

# FROM JSON FILE

In [ ]:
# properties('eval_path')

# import pandas as pd

In [ ]:
# Path to json
dataset = 'UNR_SPLICE'
json_file_name = dataset + '_FILL.json'
json_file = properties('eval_path') + json_file_name 
json_file

In [ ]:
# Read the json
import json

with open(json_file) as data_file:    
    data = json.load(data_file)
    
# http://stackoverflow.com/questions/5316720/python-how-to-convert-a-list-of-dictionaries-values-into-int-float-from-string

In [ ]:
# Convert lat & long to float
# [dict([a, int(x)] for a, x in b.iteritems()) for b in list]
for dictionary in data['stations']:
    dictionary['lat'] = float(dictionary['lat'])
    dictionary['long'] = float(dictionary['long'])

In [ ]:
import pymongo
from pymongo import MongoClient

In [ ]:
client = MongoClient('localhost', 27017)

In [ ]:
database_name=dataset+'_timeseries_database'
database_name

In [ ]:
# Delete existing database
client.drop_database(database_name)

In [ ]:
# Create new database
db = client.gps_timeseries_database
db =client[database_name]

In [ ]:
# Create 2 collections
# for meta
collections_meta = db.collections_meta
# for stations
collections_stations = db.collections_stations 

In [ ]:
meta= {}
meta['update_time'] = data['update_time']
meta['data_source'] = data['data_source']
meta['begin_date'] = data['begin_date']
meta['end_date'] = data['end_date']
meta['center_longitude'] = data['center_longitude']
meta['center_latitude'] = data['center_latitude']
meta['server_url'] = data['server_url']
meta['stateChangeNumTxtFile'] = data['stateChangeNumTxtFile']
meta['stateChangeNumJsInput'] = data['stateChangeNumJsInput']
meta['allStationInputName'] = data['allStationInputName']
meta['Filters'] = data['Filters']
meta['video_url'] = data['video_url']
meta['station_count'] = data['station_count']
collections_meta.insert_one(meta)

In [ ]:
# Add station data
collections_stations.insert_many(data['stations'])

# Create index based on latitude, longitude
db.collections_stations.create_index( [("lat", pymongo.ASCENDING), \
                                     ("lon", pymongo.ASCENDING)] )

### Query to get network information

In [ ]:
output = {}
cursor_meta = collections_meta.find()

for i in cursor_meta:
    i.pop('_id', None)
    output = i;
    
    
print output
#     i.pop('_id', None)
#     print i

### Get status of stations within bounding box 

In [ ]:
# {"age" : {"$gte" : 18, "$lte" : 30}}
# cursor = collections_stations.find( {"lat" : {"$gte" : 30, "$lte" : 60}, \
#                                      "long" : {"$gte" : -170, "$lte" : -80}})

lat_min = 30
lat_max = 60
long_min = -170
long_max = -80
cursor_stations = collections_stations.find( {"lat" : {"$gte" : lat_min, "$lte" : lat_max}, \
                            "long" : {"$gte" : long_min, "$lte" : long_max}})

if cursor_stations:
    list_station = []
    for i in cursor_stations:
        i.pop('_id', None)
        list_station.append(i)
            

In [ ]:
df_res = pd.DataFrame(list_station)
df_res

In [ ]:
df_res.columns

In [ ]:
print str(df_stations["lat"].min()) + " : " + str(df_stations["lat"].max())
print str(df_stations["long"].min()) + " : " + str(df_stations["long"].max())


# -89.998914447 : 83.64323665
# -359.998835383 : -0.037155605

In [ ]:
outputPath = os.path.join(os.getcwd(), "test_output.json")
outputPath

In [ ]:
with open(outputPath, 'w') as jsonfile:
    jsonfile.write(json.dumps(output, sort_keys=True, indent=2))

In [ ]:
jsonfile.close()
client.database_names()

In [ ]:
# client.drop_database(database_name)